In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

import torch
torch.manual_seed(42)
from google.colab import drive
drive.mount('/content/drive')

data_path = 'drive/MyDrive/MLE/protein_data/'

GPU = True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_parquet(data_path + 'train_protbert_embeddings.parquet').drop(columns=['numpy_embeddings'])
df.head(1)

,family_id,sequence_name,family_accession,aligned_sequence,sequence
8,NBD_C,R5HG95_9SPIR/261-424,PF17042.5,FVVSGSATQITANQIEKLEEADEF.ENL.LVI......N.LDMKTV...,FVVSGSATQITANQIEKLEEADEFENLLVINLDMKTVLAGVSDEIV...


In [5]:
# Load protbert model
!pip install transformers
from transformers import BertForMaskedLM, BertTokenizer, pipeline
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
if GPU:
    model.cuda()
import torch
import re


x_aminos = re.compile("[UZOB]")

def prepare_input(seq, **tokenizer_args):
    seq = x_aminos.sub("X", ' '.join(seq))
    input_ids = tokenizer.encode(seq, add_special_tokens=True, **tokenizer_args)
    return input_ids

def get_embeddings(seq, **tokenizer_args):
    input_ids = prepare_input(seq, **tokenizer_args)
    input_ids = torch.tensor([input_ids], device='cuda' if GPU else 'cpu')
    with torch.no_grad():
        return model(input_ids)

eg_input = get_embeddings("RRWWRRRRW")
eg_input

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.3 MB/s 
     |████████████████████████████████| 182 kB 64.5 MB/s 
     |████████████████████████████████| 7.6 MB 65.4 MB/s 


Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MaskedLMOutput(loss=None, logits=tensor([[[-1.8532e+01, -1.9899e+01, -1.9202e+01, -1.9233e+01, -2.1818e+01,
          -3.3183e-01, -4.3616e-01, -1.6149e-01, -2.5575e-02, -1.2855e+00,
           4.6538e-01, -1.3698e+00,  2.5464e-02,  1.3961e+00, -2.3760e+00,
          -2.5161e-01, -6.2315e-01, -9.2690e-01, -8.8102e-01, -1.4678e+00,
          -1.6217e+00,  3.2145e+00, -2.0604e+00, -4.5068e-01,  4.5484e-01,
          -3.9251e+00, -1.8108e+01, -1.8154e+01, -1.8443e+01, -1.8931e+01],
         [-2.1589e+01, -2.1636e+01, -2.1419e+01, -2.1296e+01, -2.2876e+01,
          -8.4694e-01, -8.1241e-01,  7.8902e-01, -1.7455e+00, -1.4653e+00,
           5.6490e-01, -1.1674e+00,  1.2356e+00,  4.1438e+00, -2.4756e+00,
           2.0495e+00, -1.3056e+00,  7.8861e-01,  1.2413e+00, -2.7057e+00,
          -1.4608e+00, -3.3680e-01, -2.7666e+00,  2.4256e-01,  3.9161e+00,
          -6.9252e+00, -2.1043e+01, -2.1170e+01, -2.0820e+01, -2.1201e+01],
         [-2.1355e+01, -2.0936e+01, -2.0803e+01, -1.9996e+01, -2.

In [6]:
## import libraries

#PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
from torchvision import models
from torch.nn import init
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


#other
from IPython.core.debugger import set_trace
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
import math 
from google.colab import files
import io
import datetime as dt
import pandas_profiling
import pandas_profiling as pp
from math import sqrt

#graphs
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import matplotlib.dates as dates
import seaborn as sns
import pylab
import matplotlib
import matplotlib.dates
from IPython.display import display
import plotly.graph_objects as go


%matplotlib inline


# load tqdm
from tqdm import tqdm, tqdm_notebook, tnrange

device = torch.device("cuda:0" if GPU else "cpu")
device

device(type='cuda', index=0)

In [7]:
# How to do padding + batching?
max_length = 100
in_1 = prepare_input("RRWWRRRRW", padding='max_length', max_length=max_length)
in_2 = prepare_input("NDCQ", padding='max_length', max_length=max_length)
input_ids = torch.tensor([in_1, in_2], device='cuda' if GPU else 'cpu')
with torch.no_grad():
    eg_batch_input = model(input_ids)
eg_batch_input[0].shape

torch.Size([2, 100, 30])

In [9]:
# Create a special dataloaders thing that does the prepare_input func
import torch
from torch.utils.data import Dataset

class ProteinSequenceDataset(Dataset):
    def __init__(self, df, sequence_col='sequence', label_col='family_id', max_len=100):
        self.df = df.reset_index(drop=True)
        self.sequence_col = sequence_col
        self.label_col = label_col
        self.max_len = max_len
        self._label_translator = {l: i for i, l in enumerate(sorted(df[label_col].unique()))}

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        seq, label = row[self.sequence_col], row[self.label_col]
        # Convert to tensor
        seq = prepare_input(seq[:self.max_len-10], padding='max_length', max_length=self.max_len)
        seq = torch.tensor(seq, device='cuda' if GPU else 'cpu')
        label = self._label_translator[label]
        return seq, label

In [10]:
df['family_code'] = df['family_id'].cat.codes
small_df = df[df['family_id'].isin(df['family_id'].value_counts()[:3].index)]
train_df = small_df.sample(frac=0.8)
test_df = small_df.drop(train_df.index)

In [11]:
train_dataset = ProteinSequenceDataset(train_df, label_col='family_code')
test_dataset  = ProteinSequenceDataset(test_df, label_col='family_code')
train_dataset

In [12]:
from torch.utils.data import DataLoader

batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [13]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels

(tensor([[ 2, 10,  5,  ...,  0,  0,  0],
         [ 2, 17, 15,  ...,  0,  0,  0],
         [ 2,  6,  5,  ...,  0,  0,  0],
         ...,
         [ 2,  7,  8,  ...,  0,  0,  0],
         [ 2,  9,  5,  ...,  0,  0,  0],
         [ 2,  6,  5,  ...,  0,  0,  0]], device='cuda:0'),
 tensor([1, 0, 2, 0, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 0, 2, 0, 0, 2, 1, 2,
         0, 1, 2, 0, 1, 1, 0, 2, 0, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 2, 1, 2, 0, 2,
         2, 2, 1, 1, 1, 2, 0, 1, 1, 2, 2, 2, 1, 1, 1, 2]))

In [14]:
for i in range(len(train_dataset)):
    x = train_dataset[i][0].shape
    if x[0] > 100:
        print(i)
        break

In [47]:
# Create a nn class that does lstm and batching
class LSTMClassifier(nn.Module):
    """From https://stackoverflow.com/questions/47952930/how-can-i-use-lstm-in-pytorch-for-classification"""

    def __init__(self, hidden_dim, label_size, num_layers):
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        super().__init__()
        # NN part
        self.word_embeddings = model
        self.lstm = nn.LSTM(30, hidden_dim, num_layers, bidirectional=True, batch_first=True, dropout=0.2)
        self.hidden2label = nn.Linear(hidden_dim, label_size)

    def forward(self, sentence):
        with torch.no_grad():
            embeds = self.word_embeddings(sentence)[0]
            x = embeds[:,-10:,:] # Only need last ten
        lstm_out, (ht, ct) = self.lstm(x)
        y  = self.hidden2label(ht[-1])
        log_probs = F.log_softmax(y, dim=-1)
        return log_probs

In [48]:
# i, data = next(enumerate(train_dataloader))
# # Every data instance is an input + label pair
# inputs, labels = data

# # Zero your gradients for every batch!
# optimizer.zero_grad()

# # Make predictions for this batch
# with torch.no_grad():
#     outputs = lstm(inputs)
# outputs

In [49]:
lstm = LSTMClassifier(20, 3, 3)
if GPU:
    lstm.cuda()

loss_fn = torch.nn.CrossEntropyLoss()
params_to_modify = list(lstm.lstm.parameters()) + list(lstm.hidden2label.parameters())  # + list(lstm.hidden)
optimizer = torch.optim.Adam(params_to_modify, lr=0.01)


def train_one_epoch(epoch_index, verbosity=10):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(train_dataloader) instead of
    # iter(train_dataloader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (inputs, labels) in enumerate(train_dataloader):
        if GPU:
            labels = labels.cuda()

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = lstm(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % verbosity == verbosity-1:
            print('  batch {} loss: {:,.8f}'.format(i + 1, running_loss / ((i+1)*batch_size) ))
    print(f'  total avg loss: {running_loss/ ((i+1)*batch_size):,.6f}')
    return last_loss

In [50]:
# train_one_epoch(0, verbosity=1)

In [44]:
def test_one_epoch(verbosity=10):
    with torch.no_grad():
        running_vloss = 0.0
        correct = 0
        for i, (vinputs, vlabels) in enumerate(test_dataloader):
            if GPU:
                vlabels = vlabels.cuda()
            # if vlabels.shape[0] < batch_size:
            #     print('Out of batches, exiting')
            #     break
            voutputs = lstm(vinputs)
            for voutvec, vcor in zip(voutputs, vlabels):
                vout = max([(i, v) for (i, v) in enumerate(voutvec)], key=lambda x: x[1])[0]
                if vout == vcor: correct += 1
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss
            if i % verbosity == verbosity-1:
                print(f'{i=} loss test {running_vloss/(i+1):,.6f} acc {correct / (i+1):,.6f}')
        avg_vloss = running_vloss / (batch_size*(i + 1))
        print(f'LOSS test {avg_vloss:,.6f} acc {correct / (batch_size*(i+1)):,.6f}')

In [51]:
test_one_epoch(verbosity=1000)

LOSS test 1.152226 acc 0.347656


In [52]:
EPOCHS = 50
for epoch in range(EPOCHS):
    print(f'Starting training epoch {epoch}')
    train_one_epoch(0, verbosity=30)
    print(f'Starting test epoch {epoch}')
    test_one_epoch(verbosity=1_000)

Starting training epoch 0
  batch 30 loss: 0.01082585
  batch 60 loss: 0.00955948
  batch 90 loss: 0.00888032
  total avg loss: 0.008853
Starting test epoch 0
LOSS test 0.451752 acc 0.727539
Starting training epoch 1
  batch 30 loss: 0.00740912
  batch 60 loss: 0.00642002
  batch 90 loss: 0.00560883
  total avg loss: 0.005577
Starting test epoch 1
LOSS test 0.249263 acc 0.837891
Starting training epoch 2
  batch 30 loss: 0.00342096
  batch 60 loss: 0.00305124
  batch 90 loss: 0.00306227
  total avg loss: 0.003068
Starting test epoch 2
LOSS test 0.191210 acc 0.867188
Starting training epoch 3
  batch 30 loss: 0.00254605
  batch 60 loss: 0.00275659
  batch 90 loss: 0.00273477
  total avg loss: 0.002748
Starting test epoch 3
LOSS test 0.173747 acc 0.880859
Starting training epoch 4
  batch 30 loss: 0.00248710
  batch 60 loss: 0.00245169


KeyboardInterrupt: ignored